# Progetto di Foundations of Computer Science - 2019

####  Importo le librerie necessarie

In [3]:
import pandas as pd
import numpy
import re

##### Leggo i file CSV dal PC di Matteo

In [4]:
ll = pd.read_csv(r"/home/mcampironi/Documenti/Data Science/Foundations of CS/Dataset/loans_lenders.csv")

In [3]:
loans = pd.read_csv(r"/home/mcampironi/Documenti/Data Science/Foundations of CS/Dataset/loans.csv")

####  1. Normalize the *loan_lenders* table. In the normalized table, each row must have one *loan_id* and one *lender*.

In [5]:
df = (ll.set_index(['loan_id']).apply(lambda x: x.str.split(',').explode()).reset_index())

####  2. For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing

In [6]:
loans['duration'] = (pd.to_datetime(loans.planned_expiration_time)-pd.to_datetime(loans.disburse_time)).dt.days

NameError: name 'loans' is not defined

#### 3. Find the lenders that have funded at least twice.

In [11]:
fund = df.groupby(['lenders']).size().reset_index()
fund.columns = ['lenders', 'num_of_fund']

In [23]:
fund[fund['num_of_fund'] >= 2]

,lenders,num_of_fund
0,000,39
1,00000,39
2,0002,70
4,0101craign0101,71
5,0132575,4
...,...,...
1639018,zyrorl,3
1639020,zzaman,11
1639021,zzanita,2
1639024,zzmcfate,56
